In [ ]:
import re
import numpy
import pandas
from matplotlib import pyplot

In [ ]:
filename = "dft.out"
f = open(filename, "r")
data = f.readlines()
f.close()

In [ ]:
# check method
start = data.index(" Gaussian 09:  EM64W-G09RevD.01 13-Apr-2013\n") + 3
stop = data.index(" ----------------------------------\n") + 3
print("".join(data[start:stop]), data[-3])

In [ ]:
# energies
buff = [s for s in data if "SCF Done:  E(RB3LYP) =" in s]
buff = "".join(buff)
buff = re.findall("-" + r"\d+" + "." + r"\d+", buff)
buff = numpy.array(buff, dtype="float")
pyplot.plot(buff)
pyplot.show()
print("SCF Done:  E(RB3LYP) =")
print(buff[-1], "Hartree")
print(buff[-1] * 2625.5, "kJ mol^-1")
print("")
buff = [s for s in data if "Sum of electronic and zero-point Energies=" in s]
buff = "".join(buff)
buff = re.findall("-" + r"\d+" + "." + r"\d+", buff)
buff = numpy.array(buff, dtype="float")[0]
print("Sum of electronic and zero-point Energies =")
print(buff, "Hartree")
print(buff * 2625.5, "kJ mol^-1")

In [ ]:
# optimized structure
start = data.index("    -- Stationary point found.\n")
buff = data[start:]
stop = buff.index("                    Distance matrix (angstroms):\n")
buff = buff[:stop]
start = (buff.index("                          Input orientation:                          \n") + 5)
buff = buff[start:]
stop = buff.index(" ---------------------------------------------------------------------\n")
buff = buff[:stop]
for i in range(len(buff)): buff[i] = buff[i][10:-1] + "\n"
buff = "".join(buff)
print(buff)

In [ ]:
# IR spectrum
buff = [s for s in data if "Frequencies" in s]
buff = "".join(buff)
x = re.findall(r'([+-]?[0-9]+\.?[0-9]*)', buff)
x = numpy.round(numpy.array(x, dtype="float") * 0.9613, 2)

buff = [s for s in data if "IR Inten" in s]
buff = "".join(buff)
y = re.findall(r"\d+" + "." + r"\d+", buff)
y = numpy.round(numpy.array(y, dtype="float"), 2)

pyplot.xlim(4000, 0)
pyplot.bar(x, y, width=30)
pyplot.show()
spec = pandas.Series(y, index=x)
# spec.to_csv("spec.csv")
print(spec)

In [ ]:
def g(px, py):
    w = 15
    gauss = py * numpy.exp(-4 * numpy.log(2) * (px - x) ** 2 / w ** 2)
    return gauss

In [ ]:
x = numpy.arange(0, 4001, 1)
y = x * 0
for i in range(len(spec)): y = y + g(spec.index[i], spec.values[i])
pyplot.xlim(4000, 0)
pyplot.plot(x, y)
pyplot.show()